In [1]:
from cuquantum import Network, CircuitToEinsum, contract, contract_path
import cupy as cp


def run_cutensor(circuit: QuantumCircuit):
    myconverter = CircuitToEinsum(circuit, dtype="complex128", backend=cp)
    pauli_string = "Z" * circuit.num_qubits
    expression, operands = myconverter.expectation(pauli_string, lightcone=True)

    with Network(expression, *operands) as tn:
        start = perf_counter()
        path, info = tn.contract_path()
        tn.autotune(iterations=5)
        compile_time = perf_counter() - start

        print(info)

        start = perf_counter()
        result = tn.contract()
        exec_time = perf_counter() - start

    return result, {
        "cutensor_compile": compile_time,
        "cutensor_exec": exec_time,
        "cutensor_cost": info.opt_cost,
    }

/home/di75piq/qtpu/.venv/lib/python3.11/site-packages/cupy/_environment.py:369: UserWarning: CuPy failed to preload library (/home/di75piq/qtpu/.venv/lib/python3.11/site-packages/cutensor/lib/libcutensor.so.2): OSError (libcublasLt.so.11: cannot open shared object file: No such file or directory)
  warnings.warn(msg)


NameError: name 'QuantumCircuit' is not defined

In [ ]:
from benchmark.ansatz import (
    generate_ansatz,
    qaoa,
    generate_seperator_graph,
    generate_clustered_graph,
)


edges = generate_seperator_graph(5, 15, 3)
# edges = generate_clustered_graph(3, 4, 2)

circuit = qaoa(edges, 2)
# print(circuit)

result, info = run_cutensor(circuit)
print(result)
print(info)